In [1]:
# Python setup
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from pathlib import Path
from torchvision import transforms
from torchvision import datasets
from torch.utils.data import DataLoader, Sampler, Subset
from torchvision import models
import torch.nn as nn
from torch import optim, cuda
import torch
from datetime import datetime
import pandas as pd

In [2]:
# Setting up paths
base_path = Path('./clean_data/').absolute()
raw_base_path = base_path / 'motorcycles'
raw_train_path = str(raw_base_path / 'train')
raw_val_path = str(raw_base_path / 'val')
raw_test_path = str(raw_base_path / 'test')

In [3]:
def get_model(num_classes):
    '''
    Creates a resnet-50 pretrained model and replaces the classifier with a new classifier
    '''
    model = models.resnet34(pretrained=True)

    for param in model.parameters():
        param.requires_grad = False

    num_inputs = model.fc.in_features
    model.fc = nn.Sequential(nn.Linear(num_inputs, 256),
                                       nn.ReLU(),
                                       # Get rid of dropout. I will re-evaluate later
                                       #nn.Dropout(0.4),
                                       nn.Linear(256, num_classes),
                                       nn.LogSoftmax(dim=1))
    # Move to the GPU
    model = model.to('cuda')
    return model

In [4]:
def forward_pass(model, dataloader, criterion, num_images, clear_cuda_cache=True):
    '''
    Performs a forward pass getting loss and accuracy, without modifying the model
    model: A pytorch NN 
    data A pytorch Dataloader
    clear_cuda_cache: Do we want to clear cuda memory when possible?
    num_images: Int representing the number of images being processed. This is needed because
                a sampler might return fewer results than the total dataset.
    '''
    total_loss = 0
   
    with torch.no_grad():
        model.eval()
        
        for data, target in dataloader:
            data = data.to('cuda')
            target = target.to('cuda')
            result = model(data)
            loss = criterion(result, target)
            batch_loss = loss.item() * data.size(0)
            total_loss += batch_loss
            if clear_cuda_cache is True:
                data = None
                target = None
                cuda.empty_cache()
    mean_loss = total_loss / num_images
    return({'mean_loss': mean_loss})

In [5]:
def get_accuracy(model, dataloader, num_images, clear_cuda_cache=True):
    '''
    Performs a forward pass getting loss and accuracy, without modifying the model
    model: A pytorch NN 
    data A pytorch Dataloader
    clear_cuda_cache: Do we want to clear cuda memory when possible?
    num_images: Int representing the number of images being processed. This is needed because
                a sampler might return fewer results than the total dataset.
    '''
    
    correct_predictions = 0
    with torch.no_grad():
        model.eval()
        
        for data, target in dataloader:
            data = data.to('cuda')
            target = target.to('cuda')
            result = model(data)
            _, predicted = torch.max(result.data, 1)
            # Get accurate images
            correct_predictions += (predicted == target).sum().item()
            if clear_cuda_cache is True:
                data = None
                target = None
                cuda.empty_cache()
    return correct_predictions / num_images

In [6]:
# See https://machinelearningmastery.com/adam-optimization-algorithm-for-deep-learning/ for hyperparameters
# hyper parameters are set to the research recommendations
def basic_train_model(dataloaders, epochs, clear_cuda_cache=True, name='basic model',
                      alpha=.001, beta1=0.9, beta2=0.999, epsilon=10e-8, weight_decay=0):
    '''
    Very-early training function. Not much here, but the basics to train the
    model and report loss and cuda memory
    dataloader: A Pytorch dataloader with train, validation and test datasets. All dataloaders
                should have the same number of classes
    clear_cuda_cache: Boolean telling us to clear the cuda cache when possible
    name: String with a name to give the model.
    '''
    all_start_time = datetime.now()
    results = []
    cuda_memory = []
    
    # The model changes with the number of classes, so we need to get that number.
    # A dataset with a sampler, may not include all classes in the dataset, so we
    # need to iterate to find the distinct classes
    # We also need the number of images, with the same constraints as classes. We
    # will calculate it here for accuracy
    included_classes = []
    num_images = 0
    print('Gathering configurations')
    config_start_time = datetime.now()
    for item in dataloaders['train']:
        included_classes = included_classes + item[1].tolist()
        num_images += len(item[1])
    num_classes = len(set(included_classes))
    
    # Get the number of val images returned in the dataloader. It could be a subset of the dataset if a sampler
    # is used
    num_val_images = 0
    for item in dataloaders['val']:
        num_val_images += len(item[1])
        
    # Get the number of images returned in the dataloader. It could be a subset of the dataset if a sampler
    # is used
    num_test_images = 0
    for item in dataloaders['test']:
        num_test_images += len(item[1])
    config_end_time = datetime.now()
    print(f"Configuration: {config_end_time - config_start_time}")
    
    print('Preparing model')
    model_start_time = datetime.now()
    model = get_model(num_classes)

    criterion = nn.NLLLoss()
    optimizer = optim.Adam(model.parameters(), lr=alpha, betas=(beta1, beta2), 
                           eps=epsilon, weight_decay=weight_decay)
    model_end_time = datetime.now()
    print(f'Model preparation: {model_end_time - model_start_time}')
    for epoch in range(epochs):
        epoch_start = datetime.now()
        print(f'Epoch: {epoch + 1}')
        train_loss = 0.0
        for data, targets in dataloaders['train']:
            #Get cuda memory
            cuda_memory.append({
                'name': name,
                'timestamp': datetime.now(),
                'cuda_memory': cuda.memory_allocated()})
            data = data.to('cuda')
            targets = targets.to('cuda')
            cuda_memory.append({
                'name': name,
                'timestamp': datetime.now(),
                'cuda_memory': cuda.memory_allocated()})
            # Clear the gradients
            optimizer.zero_grad()
            out = model(data)
            loss = criterion(out, targets)
            loss.backward()
            # Get loss for the batch
            batch_loss = loss.item() * data.size(0)
            train_loss += batch_loss
            optimizer.step()
            #Get cuda memory
            cuda_memory.append({
                'name': name,
                'timestamp': datetime.now(),
                'cuda_memory': cuda.memory_allocated()})
            # Clear the batch from cuda memory. It is no longer needed
            if clear_cuda_cache is True:
                data = None
                targets = None
                cuda.empty_cache()
        
        mean_train_loss = train_loss / num_images

        # Get train accuracy to see if the model is learning at all
        #train_accuracy = get_accuracy(model, dataloaders['train'], num_images)
        # Get validation loss
       
        validation_results = forward_pass(model, dataloaders['val'], criterion, num_images=num_val_images,
                                          clear_cuda_cache=clear_cuda_cache)
        # Get test accuracy
        test_accuracy = get_accuracy(model, dataloaders['test'], num_test_images)
        epoch_end = datetime.now()
        print(f'Epoch run time: {epoch_end - epoch_start}, Train_loss: {mean_train_loss}, Val loss: {validation_results["mean_loss"]}, Test Accuracy: {test_accuracy}')
        results.append({
            'epoch': epoch + 1,
            'epoch_runtime': epoch_end - epoch_start,
            'train_loss': mean_train_loss,
            'val_loss': validation_results['mean_loss'],
            'test_accuracy': test_accuracy})
        
    all_end_time = datetime.now()
    return {'model': model, 'name': name, 'results': results, 'cuda_memory': cuda_memory, 
            'run_time': all_end_time - all_start_time, 'config_run_time': config_end_time - config_start_time,
            'model_run_time': model_end_time - model_start_time}

In [7]:
batch_size = 256

basic_transforms = {
    'train': transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        # Normalize using same mean, std as imagenet
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.Resize(size=512),
        transforms.CenterCrop(size=448),
        transforms.ToTensor(),
        # Normalize using same mean, std as ResNet
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}
basic_data = {
    'train': datasets.ImageFolder(root=raw_train_path, transform = basic_transforms['train'] ),
    'valid': datasets.ImageFolder(root=raw_val_path, transform = basic_transforms['valid']),
    'test': datasets.ImageFolder(root=raw_test_path, transform = basic_transforms['valid'])
}

basic_dataloaders = {
    'train': DataLoader(basic_data['train'], batch_size=batch_size, shuffle=True),
    'val': DataLoader(basic_data['valid'], batch_size=batch_size, shuffle=True),
    'test': DataLoader(basic_data['test'], batch_size=batch_size, shuffle=True),
}



In [8]:
class TargetSampler(Sampler):
    '''
    Sampler base on the number of targets we want. Rather than random sampling from all 
    targets, which may not have the same targets as test and train, we pull all images from 
    the first x number of targets
    
    This was extremely slow at first, because I was iterating through the data instead of 
    just the targets. Now it takes no noticable time

    '''
    def __init__(self, data, num_targets):
        '''
        data: A pytorch dataset (In this case, an ImageFolder)
        num_targets: Int representing the first X targets to use.
        '''
        self.num_targets = num_targets
        self.data = data
    
    def __iter__(self):
        indices = []
        for index, target in enumerate(self.data.targets):
            # Add the indice if the target is in range
            if target < self.num_targets:
                indices.append(index)
        return iter(indices)
    
    def __len__(self):
        # Count the number of images in the target range
        # The image is a tuple, with image[1] being the target
        return len([i for i in self.data.targets if i < self.num_targets])


In [14]:
num_targets = 50
num_epochs = 5
batch_size = 128

In [15]:
train_sampler = TargetSampler(basic_data['train'], num_targets=num_targets)
val_sampler = TargetSampler(basic_data['valid'], num_targets=num_targets)
test_sampler = TargetSampler(basic_data['test'], num_targets=num_targets)

high_res_basic = {
    'train': DataLoader(basic_data['train'], batch_size=batch_size, sampler=train_sampler, shuffle=False),
    'val': DataLoader(basic_data['valid'], batch_size=batch_size, sampler=val_sampler, shuffle=False),
    'test': DataLoader(basic_data['test'], batch_size=batch_size, sampler=test_sampler, shuffle=False)
}


basic_data_basic_model = basic_train_model(dataloaders=high_res_basic, epochs=num_epochs, alpha=0.001,
                                     name='512x512 Basic data and transforms', clear_cuda_cache=True)
print(basic_data_basic_model['run_time'])

Gathering configurations
Configuration: 0:00:27.534185
Preparing model
Model preparation: 0:00:00.981706
Epoch: 1
Epoch run time: 0:00:30.066453, Train_loss: 4.07114200592041, Val loss: 3.891035556793213, Test Accuracy: 0.056818181818181816
Epoch: 2
Epoch run time: 0:00:30.107382, Train_loss: 3.8418405437469483, Val loss: 3.8656692504882812, Test Accuracy: 0.06818181818181818
Epoch: 3


KeyboardInterrupt: 

In [ ]:
model = basic_data_basic_model['model']
with torch.no_grad():
        model.eval()
        
        for data, target in basic_data_basic_model['test']:
            data = data.to('cuda')
            target = target.to('cuda')
            result = model(data)
            _, predicted = torch.max(result.data, 1)
            # Get accurate images
            #correct_predictions += (predicted == target).sum().item()
            print(f'Predicted: {predicted}, Actual: {target}')
            data = None
            target = None
            cuda.empty_cache()

In [ ]:
#resnet34.conv1 = nn.Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
#resnet34.bn1 = nn.BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)

In [ ]:
models.resnet34()